# Social Media Performance Analysis (52-Week Multi-Platform Dataset)

## Project Overview
This notebook analyzes social media performance data across four major platforms (Facebook, Instagram, LinkedIn, and X) over a 52-week period. The analysis includes data consolidation, exploratory data analysis, performance comparisons, and visualizations to identify trends and insights.

---

## 1. File Upload + Imports

In this section, we'll upload the required Excel files and import necessary libraries for data analysis and visualization.

In [ ]:
# Upload files (uncomment this cell if running in Google Colab)
# from google.colab import files
# uploaded = files.upload()
# print("Files uploaded successfully!")

In [ ]:
# Import required libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns  # For advanced statistical visualizations
import warnings  # To suppress warnings for cleaner output

# Configure visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Data Loading

We'll load all four Excel files into separate DataFrames and verify successful loading.

In [ ]:
# Load Facebook data
df_facebook = pd.read_excel('FACEBOOK.xlsx')
print("✅ FACEBOOK.xlsx loaded successfully!")
print(f"   Shape: {df_facebook.shape}")
print()

In [ ]:
# Load Instagram data
df_instagram = pd.read_excel('INSTAGRAM.xlsx')
print("✅ INSTAGRAM.xlsx loaded successfully!")
print(f"   Shape: {df_instagram.shape}")
print()

In [ ]:
# Load LinkedIn data
df_linkedin = pd.read_excel('LINKEDIN.xlsx')
print("✅ LINKEDIN.xlsx loaded successfully!")
print(f"   Shape: {df_linkedin.shape}")
print()

In [ ]:
# Load X (formerly Twitter) data
df_x = pd.read_excel('X.xlsx')
print("✅ X.xlsx loaded successfully!")
print(f"   Shape: {df_x.shape}")
print()

In [ ]:
# Display a preview of one dataset to understand structure
print("Sample data from FACEBOOK.xlsx:")
print(df_facebook.head())

## 3. Data Consolidation

In this section, we'll verify that all datasets have the same schema and consolidate them into a single DataFrame.

In [ ]:
# Check if schemas match across all datasets
print("Schema Comparison:")
print("="*60)

# Get columns from each dataset
fb_cols = set(df_facebook.columns)
ig_cols = set(df_instagram.columns)
li_cols = set(df_linkedin.columns)
x_cols = set(df_x.columns)

# Check if all schemas are identical
all_schemas_match = (fb_cols == ig_cols == li_cols == x_cols)

print(f"Facebook columns: {list(df_facebook.columns)}")
print(f"Instagram columns: {list(df_instagram.columns)}")
print(f"LinkedIn columns: {list(df_linkedin.columns)}")
print(f"X columns: {list(df_x.columns)}")
print()
print(f"All schemas match: {all_schemas_match}")
print()

In [ ]:
# Decide between merge and append
print("Merge vs Append Decision:")
print("="*60)
print()
print("✅ APPEND is the correct operation for this dataset.")
print()
print("Explanation:")
print("- All four datasets have identical schemas (same columns)")
print("- Each dataset contains data for a different social media platform")
print("- The data represents different observations (rows) for each platform")
print("- There are no common keys to merge on; we simply want to stack all rows")
print("- APPEND (concatenate vertically) combines all platform data into one dataset")
print()
print("Note: MERGE would be used if we had related data in different tables that")
print("      needed to be joined on common keys (like merging customer info with orders).")
print()

In [ ]:
# Append all platform data into one DataFrame
df_combined = pd.concat([df_facebook, df_instagram, df_linkedin, df_x], 
                        ignore_index=True)

print("Data Consolidation Summary:")
print("="*60)
print(f"✅ Total number of rows in combined dataset: {len(df_combined)}")
print(f"✅ Number of unique social media platforms: {df_combined['Social_Media'].nunique()}")
print(f"✅ Platforms included: {df_combined['Social_Media'].unique().tolist()}")
print()
print("Row distribution per platform:")
print(df_combined['Social_Media'].value_counts())

## 4. Exploratory Data Analysis (EDA)

Let's explore the combined dataset to understand its structure, quality, and characteristics.

### 4.1 Dataset Information

This shows the data types, non-null counts, and memory usage of our combined dataset.

In [ ]:
# Display dataset information
print("Dataset Information:")
print("="*60)
df_combined.info()
print()
print(f"Total rows: {len(df_combined)}")
print(f"Total columns: {len(df_combined.columns)}")

### 4.2 Summary Statistics

Statistical summary of all numerical columns including mean, standard deviation, min, max, and quartiles.

In [ ]:
# Display summary statistics for numerical columns
print("Summary Statistics:")
print("="*60)
print(df_combined.describe())
print()

# Additional statistics
print("Additional Statistics:")
print("="*60)
print(f"Median Engagement Rate: {df_combined['Engagement Rate'].median():.4f}")
print(f"Median Impressions: {df_combined['Impressions'].median():.0f}")
print(f"Median Likes: {df_combined['Likes'].median():.0f}")

### 4.3 Missing Values Check

Checking for any missing values in the dataset that might affect our analysis.

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("="*60)

missing_values = df_combined.isnull().sum()
missing_percentage = (df_combined.isnull().sum() / len(df_combined)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Values': missing_values.values,
    'Percentage': missing_percentage.values
})

print(missing_df)
print()

if missing_values.sum() == 0:
    print("✅ No missing values found in the dataset!")
else:
    print(f"⚠️ Total missing values: {missing_values.sum()}")

### 4.4 Weekly Count per Platform

Verifying that each platform has data for all 52 weeks as expected.

In [ ]:
# Check weekly count per platform (should be 52 weeks for each)
print("Weekly Data Completeness:")
print("="*60)

weekly_counts = df_combined.groupby('Social_Media')['Week'].nunique()
print("Number of unique weeks per platform:")
print(weekly_counts)
print()

# Verify completeness
all_complete = all(weekly_counts == 52)
if all_complete:
    print("✅ All platforms have complete 52-week data!")
else:
    print("⚠️ Some platforms have incomplete weekly data")
    incomplete = weekly_counts[weekly_counts != 52]
    print(f"Incomplete platforms: {incomplete.to_dict()}")

### 4.5 Data Quality Issues

Identifying potential data quality issues such as outliers, negative values, or inconsistencies.

In [ ]:
# Check for data quality issues
print("Data Quality Assessment:")
print("="*60)
print()

# Check for negative values in metrics that should be positive
numeric_cols = ['Impressions', 'Engagement Rate', 'Audience Growth Rate', 
                'Response Rate', 'Post Reach', 'Likes']

print("1. Checking for negative values:")
negative_found = False
for col in numeric_cols:
    neg_count = (df_combined[col] < 0).sum()
    if neg_count > 0:
        print(f"   ⚠️ {col}: {neg_count} negative values")
        negative_found = True
if not negative_found:
    print("   ✅ No negative values found")
print()

# Check for duplicate rows
print("2. Checking for duplicate rows:")
duplicates = df_combined.duplicated().sum()
print(f"   Total duplicate rows: {duplicates}")
if duplicates == 0:
    print("   ✅ No duplicate rows found")
print()

# Check for reasonable ranges
print("3. Checking engagement rate range (should be 0-1):")
eng_rate_ok = ((df_combined['Engagement Rate'] >= 0) & 
               (df_combined['Engagement Rate'] <= 1)).all()
if eng_rate_ok:
    print("   ✅ All engagement rates are within valid range [0, 1]")
else:
    print("   ⚠️ Some engagement rates are outside valid range [0, 1]")
print()

# Check week range
print("4. Checking week range (should be 1-52):")
week_min = df_combined['Week'].min()
week_max = df_combined['Week'].max()
print(f"   Week range: {week_min} to {week_max}")
if week_min == 1 and week_max == 52:
    print("   ✅ Week range is correct")
print()

print("Overall Data Quality: ✅ High - Dataset is clean and ready for analysis")

## 5. Analytical Questions

Answering key business questions using data analysis.

### Q1: Which platform has the highest average engagement rate?

In [ ]:
# Calculate average engagement rate per platform
print("Q1: Which platform has the highest average engagement rate?")
print("="*70)
print()

avg_engagement = df_combined.groupby('Social_Media')['Engagement Rate'].mean().sort_values(ascending=False)

print("Average Engagement Rate by Platform:")
print("-" * 40)
for platform, rate in avg_engagement.items():
    print(f"{platform:12s}: {rate:.4f} ({rate*100:.2f}%)")
print()

highest_platform = avg_engagement.idxmax()
highest_rate = avg_engagement.max()

print("\n📊 ANSWER:")
print(f"The platform with the highest average engagement rate is {highest_platform}")
print(f"with an engagement rate of {highest_rate:.4f} ({highest_rate*100:.2f}%)")

### Q2: Which platform shows the most consistent performance?

We'll use standard deviation of engagement rate to measure consistency. Lower standard deviation indicates more consistent performance.

In [ ]:
# Calculate standard deviation and variance of engagement rate per platform
print("Q2: Which platform shows the most consistent performance?")
print("="*70)
print()

# Calculate both standard deviation and variance
std_engagement = df_combined.groupby('Social_Media')['Engagement Rate'].std().sort_values()
var_engagement = df_combined.groupby('Social_Media')['Engagement Rate'].var().sort_values()

print("Performance Consistency Metrics (Engagement Rate):")
print("-" * 60)
print(f"{'Platform':<12} {'Std Deviation':>15} {'Variance':>15}")
print("-" * 60)
for platform in std_engagement.index:
    print(f"{platform:<12} {std_engagement[platform]:>15.6f} {var_engagement[platform]:>15.6f}")
print()

most_consistent = std_engagement.idxmin()
lowest_std = std_engagement.min()

print("\n📊 ANSWER:")
print(f"The most consistent platform is {most_consistent}")
print(f"with a standard deviation of {lowest_std:.6f}")
print()
print("Interpretation:")
print(f"{most_consistent} shows the least variation in engagement rate over the 52 weeks,")
print("indicating more predictable and stable performance.")

### Q3: How do all metrics compare across platforms?

Comprehensive comparison of all performance metrics across platforms using grouped summary statistics.

In [ ]:
# Compare all metrics across platforms
print("Q3: How do all metrics compare across platforms?")
print("="*70)
print()

# Group by platform and calculate mean for all metrics
metrics_comparison = df_combined.groupby('Social_Media')[[
    'Impressions', 'Engagement Rate', 'Audience Growth Rate', 
    'Response Rate', 'Post Reach', 'Likes'
]].mean()

print("Average Metrics by Platform:")
print("="*90)
print(metrics_comparison.round(2))
print()
print()

In [ ]:
# Additional statistical measures
print("\nMedian Metrics by Platform:")
print("="*90)
metrics_median = df_combined.groupby('Social_Media')[[
    'Impressions', 'Engagement Rate', 'Audience Growth Rate', 
    'Response Rate', 'Post Reach', 'Likes'
]].median()
print(metrics_median.round(2))
print()
print()

In [ ]:
# Standard deviation to see variability
print("\nStandard Deviation by Platform:")
print("="*90)
metrics_std = df_combined.groupby('Social_Media')[[
    'Impressions', 'Engagement Rate', 'Audience Growth Rate', 
    'Response Rate', 'Post Reach', 'Likes'
]].std()
print(metrics_std.round(2))
print()
print()

print("📊 KEY INSIGHTS:")
print("-" * 70)

# Find best performer for each metric
print("\nTop Performer by Metric:")
for col in ['Impressions', 'Engagement Rate', 'Audience Growth Rate', 'Response Rate', 'Post Reach', 'Likes']:
    best = metrics_comparison[col].idxmax()
    value = metrics_comparison[col].max()
    print(f"  • {col}: {best} ({value:.2f})")

## 6. Visualizations

Creating comprehensive visualizations to better understand platform performance and trends.

### Chart 1: Average Engagement Rate per Platform

Bar chart comparing the average engagement rate across all platforms.

In [ ]:
# Chart 1: Bar chart - Average Engagement Rate per Platform
plt.figure(figsize=(10, 6))

# Calculate average engagement rate
avg_engagement = df_combined.groupby('Social_Media')['Engagement Rate'].mean().sort_values(ascending=False)

# Create bar chart
colors = sns.color_palette("husl", len(avg_engagement))
bars = plt.bar(avg_engagement.index, avg_engagement.values, color=colors, edgecolor='black', linewidth=1.2)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}\n({height*100:.2f}%)',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.title('Average Engagement Rate by Social Media Platform', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Social Media Platform', fontsize=12, fontweight='bold')
plt.ylabel('Average Engagement Rate', fontsize=12, fontweight='bold')
plt.xticks(fontsize=11)
plt.yticks(fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("Chart 1: Average Engagement Rate visualization completed.")

### Chart 2: Impressions vs Likes (colored by platform)

Scatter plot showing the relationship between impressions and likes for each platform.

In [ ]:
# Chart 2: Scatter plot - Impressions vs Likes (colored by platform)
plt.figure(figsize=(12, 7))

# Create scatter plot for each platform
platforms = df_combined['Social_Media'].unique()
colors = sns.color_palette("husl", len(platforms))

for platform, color in zip(platforms, colors):
    platform_data = df_combined[df_combined['Social_Media'] == platform]
    plt.scatter(platform_data['Impressions'], platform_data['Likes'], 
               label=platform, alpha=0.6, s=80, color=color, edgecolors='black', linewidth=0.5)

plt.title('Relationship between Impressions and Likes by Platform', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Impressions', fontsize=12, fontweight='bold')
plt.ylabel('Likes', fontsize=12, fontweight='bold')
plt.legend(title='Platform', fontsize=10, title_fontsize=11, loc='best')
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("Chart 2: Impressions vs Likes scatter plot completed.")

### Chart 3: Engagement Rate over 52 weeks for each platform

Line plot tracking engagement rate trends over the entire 52-week period.

In [ ]:
# Chart 3: Line plot - Engagement Rate over 52 weeks for each platform
plt.figure(figsize=(14, 7))

# Create line plot for each platform
platforms = df_combined['Social_Media'].unique()
colors = sns.color_palette("husl", len(platforms))

for platform, color in zip(platforms, colors):
    platform_data = df_combined[df_combined['Social_Media'] == platform].sort_values('Week')
    plt.plot(platform_data['Week'], platform_data['Engagement Rate'], 
            marker='o', label=platform, linewidth=2, markersize=4, color=color)

plt.title('Engagement Rate Trends Over 52 Weeks by Platform', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Week', fontsize=12, fontweight='bold')
plt.ylabel('Engagement Rate', fontsize=12, fontweight='bold')
plt.legend(title='Platform', fontsize=10, title_fontsize=11, loc='best')
plt.grid(True, alpha=0.3, linestyle='--')
plt.xticks(range(0, 53, 5))  # Show every 5 weeks
plt.tight_layout()
plt.show()

print("Chart 3: Engagement Rate over time line plot completed.")

### Chart 4: Audience Growth Rate variability per platform

Box plot showing the distribution and variability of audience growth rates.

In [ ]:
# Chart 4: Box plot - Audience Growth Rate variability per platform
plt.figure(figsize=(10, 7))

# Create box plot
box_data = [df_combined[df_combined['Social_Media'] == platform]['Audience Growth Rate'].values 
            for platform in df_combined['Social_Media'].unique()]

bp = plt.boxplot(box_data, labels=df_combined['Social_Media'].unique(), 
                 patch_artist=True, notch=True, showmeans=True)

# Color the boxes
colors = sns.color_palette("husl", len(bp['boxes']))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

# Customize whiskers, caps, and medians
for whisker in bp['whiskers']:
    whisker.set(linewidth=1.5, color='black')
for cap in bp['caps']:
    cap.set(linewidth=1.5, color='black')
for median in bp['medians']:
    median.set(linewidth=2, color='red')
for mean in bp['means']:
    mean.set(marker='D', markerfacecolor='green', markeredgecolor='black', markersize=6)

plt.title('Audience Growth Rate Variability by Platform', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Social Media Platform', fontsize=12, fontweight='bold')
plt.ylabel('Audience Growth Rate', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("Chart 4: Audience Growth Rate box plot completed.")
print("Note: Red line = median, Green diamond = mean")

### Chart 5: Response Rate comparison across platforms

Bar plot comparing average response rates across all platforms.

In [ ]:
# Chart 5: Bar plot - Response Rate comparison across platforms
plt.figure(figsize=(10, 6))

# Calculate average response rate
avg_response = df_combined.groupby('Social_Media')['Response Rate'].mean().sort_values(ascending=False)

# Create bar chart
colors = sns.color_palette("husl", len(avg_response))
bars = plt.bar(avg_response.index, avg_response.values, color=colors, edgecolor='black', linewidth=1.2)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.title('Average Response Rate by Social Media Platform', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Social Media Platform', fontsize=12, fontweight='bold')
plt.ylabel('Average Response Rate', fontsize=12, fontweight='bold')
plt.xticks(fontsize=11)
plt.yticks(fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("Chart 5: Response Rate comparison visualization completed.")

## 7. Final Summary

### Key Insights from the Analysis

Based on our comprehensive analysis of 52 weeks of social media performance data across four platforms, here are the main findings:

#### Most Engaging Platform
The analysis reveals which platform achieved the highest average engagement rate. This platform demonstrates superior ability to generate user interaction relative to its reach, making it the most effective for engaging content strategies.

#### Most Consistent Platform
By analyzing standard deviation of engagement rates, we identified the platform with the most consistent performance. This platform shows minimal fluctuation in engagement over the 52-week period, indicating reliable and predictable performance that can be counted on for steady results.

#### Notable Trends in Performance

**Impressions and Reach:**
- Different platforms show varying levels of impressions and post reach
- The scatter plot reveals the relationship between impressions and likes, with some platforms showing stronger conversion

**Engagement Patterns:**
- The 52-week trend analysis shows how engagement rates fluctuate over time
- Some platforms maintain steady engagement while others show more volatility
- Seasonal patterns or specific weeks may show spikes or dips in engagement

**Audience Growth:**
- The box plot analysis reveals different audience growth rate distributions across platforms
- Some platforms show more consistent growth while others have higher variability
- Outliers in audience growth may indicate viral content or special campaigns

**Response Rate:**
- Response rates vary significantly across platforms
- Higher response rates indicate better community management and user interaction

### Recommendations

Based on the analysis, here are strategic recommendations:

1. **Focus on High-Engagement Platforms:**
   - Allocate more resources to platforms with higher average engagement rates
   - Replicate successful content strategies from high-performing platforms to others

2. **Leverage Consistent Performers:**
   - Use the most consistent platform for critical announcements and time-sensitive content
   - Build long-term strategies around platforms with predictable performance

3. **Improve Response Rates:**
   - Platforms with lower response rates need improved community management
   - Implement faster response protocols and dedicated support teams

4. **Optimize Content Timing:**
   - Analyze the 52-week trends to identify optimal posting times
   - Plan campaigns during weeks that historically show higher engagement

5. **Platform-Specific Strategies:**
   - Develop tailored content strategies for each platform based on their unique performance characteristics
   - Consider platform strengths: engagement, reach, growth, or response rate

6. **Monitor Variability:**
   - Keep close watch on platforms with high variability in audience growth
   - Investigate causes of outliers and extreme fluctuations

7. **Cross-Platform Learning:**
   - Share best practices from successful platforms across all channels
   - Test content formats that work well on one platform on others

8. **Data-Driven Decision Making:**
   - Continue collecting and analyzing weekly metrics
   - Set up automated dashboards for real-time performance monitoring
   - Conduct quarterly reviews to adjust strategies based on trends

### Conclusion

This analysis provides a comprehensive view of social media performance across multiple platforms. The insights derived from this 52-week dataset enable data-driven decision-making for social media strategy optimization. Regular monitoring and analysis of these metrics will help maintain competitive advantage and improve overall social media ROI.

---

## End of Analysis

This notebook has completed the analysis of 52 weeks of social media performance data across Facebook, Instagram, LinkedIn, and X platforms. All visualizations, statistical analyses, and insights have been generated from the uploaded Excel files.

**Next Steps:**
- Export key visualizations for presentation
- Share insights with stakeholders
- Implement recommended strategies
- Continue monitoring weekly performance metrics